### Imports and config

In [11]:
# Libraries
import os
from importlib import reload
from typing import List

import tensorflow as tf
import keras
import numpy as np
import sklearn.model_selection

# Scripts
from helpers import data, runners, utils

In [12]:
!pip show tensorflow

Name: tensorflow
Version: 2.13.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /Users/antrikshdhand/Documents/github/thesis-ml/ml/virt/lib/python3.9/site-packages
Requires: tensorflow-macos
Required-by: 


In [13]:
!pip show keras

Name: keras
Version: 2.13.1
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /Users/antrikshdhand/Documents/github/thesis-ml/ml/virt/lib/python3.9/site-packages
Requires: 
Required-by: tensorflow-macos


In [14]:
reload(data)
reload(runners)
reload(utils)

<module 'helpers.utils' from '/Users/antrikshdhand/Documents/github/thesis-ml/ml/main/helpers/utils.py'>

In [15]:
# Silence information messages from TensorFlow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

# Check whether tf is using the GPU
# utils.check_gpu_use() 

## `main`

### Load data

In [16]:
SPECS_DIR = "../data/image_segmentation/data/spectrograms"
MASKS_DIR = "../data/image_segmentation/data/masks"

IMAGE_DIMS = (256, 256)
MODEL_INPUT_SIZE = (256, 256, 1)
DATA_BATCH_SIZE = 16
GPU_BATCH_SIZE = 16
EPOCHS = 2

In [17]:
def clean_dirs(specs_dir, masks_dir):
    # Get sorted lists of image and mask files
    spec_files = sorted([f for f in os.listdir(specs_dir) if f.endswith('.png')])
    mask_files = sorted([f for f in os.listdir(masks_dir) if f.endswith('.png')])

    # Keep only spec_files that have a corresponding mask
    mask_ids = {f.replace('_mask.png', '') for f in mask_files}
    spec_files = [f for f in spec_files if f.replace('.png', '') in mask_ids]

    return spec_files, mask_files

In [18]:
spec_files, mask_files = clean_dirs(SPECS_DIR, MASKS_DIR)
train_specs, test_specs, train_masks, test_masks = \
    sklearn.model_selection.train_test_split(spec_files, mask_files, test_size=0.2, random_state=42)

train_gen = data.ImageSegmentationGenerator(
    SPECS_DIR, MASKS_DIR, 
    train_specs, train_masks, 
    batch_size=DATA_BATCH_SIZE, 
    image_dims=IMAGE_DIMS, 
    shuffle=True
)

test_gen = data.ImageSegmentationGenerator(
    SPECS_DIR, MASKS_DIR, 
    test_specs, test_masks, 
    batch_size=DATA_BATCH_SIZE, 
    image_dims=IMAGE_DIMS, 
    shuffle=True
)

### Configure model

In [19]:
from models import unet

NUM_CLASSES = 1

model = unet.get_unet_model(input_size=MODEL_INPUT_SIZE, classes=NUM_CLASSES, dropout=0.1)

metrics = [
    keras.metrics.BinaryAccuracy(), 
    keras.metrics.BinaryIoU(target_class_ids=[1], threshold=0.5)
]
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=metrics)

In [ ]:
# Train the model
history = model.fit(
    train_gen,
    epochs=EPOCHS,
    batch_size=GPU_BATCH_SIZE,
    verbose=2
)

Epoch 1/2


In [ ]:
evals = model.evaluate(
    test_gen,
    batch_size=GPU_BATCH_SIZE,
    verbose=2
)